# Realiza a transformação do dataset
Extrai os seguintes dados 'data', 'relator', 'orgao', 'recorrente', 'dispositivo', 'qtd_recorrentes'

## Importação de bibliotecas básicas

In [1]:
import pandas as pd
import time

## Carregamento do dataset

In [2]:
pasta = "E:\\OneDrive\\Documentos\\pesquisa_codigos_2020\\pesquisa_jurimetria_polaridade_final\\2_Compreensao_dos_dados\\2_1_coleta_de_dados\\Datasets_coletados\\TRT4\\"
arquivo = "TRT4_inteiro_teor_2017_2018_2019_amostra.csv"
df = pd.read_csv( pasta+arquivo, encoding='utf-8' )
print( df.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29893 entries, 0 to 29892
Data columns (total 2 columns):
LINK            29892 non-null object
INTEIRO_TEOR    29893 non-null object
dtypes: object(2)
memory usage: 467.2+ KB
None


In [3]:
#df = df.drop( "_type", 1)
#df.columns = [ "LINK", "INTEIRO_TEOR" ]
#df.info()

In [4]:
df.head()

,LINK,INTEIRO_TEOR
0,rest/cache/acordao/pje/Lj4WHiWFHCG9b-Ou2xsPug?,Acórdão: 0021543-62.2015.5.04.0013 ( ROT )...
1,rest/cache/acordao/pje/diVCcAlkbQxVq599O2j4Rw?,Acórdão: 0020791-38.2016.5.04.0601 ( ROT )...
2,rest/cache/acordao/pje/ltj3zNWxiMy8bTO7ihrQRg?,Acórdão: 0021604-08.2015.5.04.0405 ( ROT )...
3,rest/cache/acordao/pje/kPKk0bNuWrWpWliK_6iAIQ?,Acórdão: 0020306-43.2018.5.04.0512 ( RORSu...
4,rest/cache/acordao/pje/3GNICpoVGGtgk67szCULAw?,Acórdão: 0020918-26.2017.5.04.0282 ( ROT )...


## Definição de funções para extrair os dados

In [5]:
import re

def extrai_data( texto ):
    try:
        result = re.search( r"(\d{1,2}\/\d{1,2}\/(\d{4}|\d{2}))", str( texto ) )
        return result[1]
    except:
        return pd.np.nan

def extrai_relator( texto ):
    try:
        regex1 = r"relator:|redator:"
        achou = re.search(regex1, texto.lower() )
        inicio = achou.end()

        regex2 = r"ementa|órgão|poder"
        achou = re.search(regex2, texto[inicio:].lower())
        fim = achou.start()
        return texto[inicio:inicio+fim].strip()
    except:
        return pd.np.nan

def extrai_orgao( texto ):
    try:
        regex1 = r"órgão julgador:|órgão julgador:"
        achou = re.search( regex1, texto.lower() )
        inicio = achou.end()

        regex2 = r"data:|relator|redator"
        achou = re.search( regex2, texto[inicio:].lower() )
        fim = achou.start()

        orgao = texto[inicio:inicio+fim].strip()
        return orgao
    except:
        return pd.np.nan
    
def extrai_recorrente(texto):
    try:
        regex1 = r"recorrentes:|recorrente:|agravante:|agravantes:|embargante:|embargantes:"
        achou = re.search( regex1, texto.lower() )
        inicio = achou.end()

        #regex2 = r"recorridos:|recorrido:|agravado:|agravados:|agravada:|agravadas:|recorridas:|recorrida:|representante:|terceiro interessado:|relator:|redator:"
        regex2 = r"recorridos|recorrido|agravado|agravados|agravada|agravadas|recorridas|recorrida|representante|terceiro interessado|relator|redator|embargado|embargados"
        achou = re.search( regex2, texto[inicio:].lower() )
        fim = achou.start()

        recorrente = texto[inicio:inicio+fim].strip()
        return recorrente
    except:
        return pd.np.nan

def extrai_dispositivo(texto):
    try:
        regex1 = r"ACORDAM|ACÓRDÃO|ACORDAO"
        achou = re.search( regex1, texto ) #Sem o .lower() pq ta sempre em maiúsculo
        
        if achou == None: #se nao achar, maiúsculo, tenta minúsculo
            regex1 = r"acordam|acórdão|acordao|fundamentos pelos quais"
            achou = re.search( regex1, texto.lower() )        
        
        inicio = achou.end()

        regex2 = r"fundamentos pelos quais"
        achou = re.search( regex2, texto[inicio:].lower() )
        if achou != None:  #se ja achou o inicio, e achou outra palavra fundamentos, vai ate o final
            return texto[inicio:].strip()
        
        regex2 = r"relatório|fundamentos"
        achou = re.search( regex2, texto[inicio:].lower() )
        if achou != None:
            fim = achou.start()
            return texto[inicio:inicio+fim].strip()
        else:
            return texto[inicio:].strip()
    except:
        return pd.np.nan
    
def quantidade_recorrentes( texto ):
    try:
        s = texto.split(",")
        if len( s ) == 1 and len( texto ) > 30: # tem nome q nao tem ",", daí se for maior q 30 considero q sao 2 nomes
            return 999
        else:
            return int( len( s ) )
    except:
        return pd.np.nan

## Extração das informações

In [6]:
%%time
df_tudo=df
#df_tudo.reset_index()
df_tudo["DATA"] = df_tudo["INTEIRO_TEOR"].map( extrai_data )
df_tudo["RELATOR"] = df_tudo["INTEIRO_TEOR"].map( extrai_relator )
df_tudo["ORGAO"] = df_tudo["INTEIRO_TEOR"].map( extrai_orgao )
df_tudo["RECORRENTE"] = df_tudo["INTEIRO_TEOR"].map( extrai_recorrente )
df_tudo["DISPOSITIVO"] = df_tudo["INTEIRO_TEOR"].map( extrai_dispositivo )
df_tudo["QTD_RECORRENTES"] = df_tudo["RECORRENTE"].map( quantidade_recorrentes )

Wall time: 25 s


In [7]:
df_tudo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29893 entries, 0 to 29892
Data columns (total 8 columns):
LINK               29892 non-null object
INTEIRO_TEOR       29893 non-null object
DATA               29892 non-null object
RELATOR            29892 non-null object
ORGAO              29892 non-null object
RECORRENTE         27555 non-null object
DISPOSITIVO        29892 non-null object
QTD_RECORRENTES    27555 non-null float64
dtypes: float64(1), object(7)
memory usage: 1.8+ MB


## Limpeza dos dados

In [8]:
#sem html e sem NAs
#apenas 1 recorrente
df_tudo = df_tudo.dropna()
df_tudo["QTD_RECORRENTES"].unique()

array([  1., 999.,   2.,   3.,  40.,   4.,   5.,  38.,   7.,   8.,  10.,
        29.,  12.,   6.,  13.,  24.,   9.,  11.,  22.,  30., 132.,  15.,
        16.,  42.,  19.,  80.,  26.,  14.,  35.])

In [9]:
df_tudo["QTD_RECORRENTES"].value_counts() 

1.0      12071
2.0       9439
999.0     4182
3.0       1189
4.0        394
5.0        128
6.0         49
7.0         22
8.0         18
11.0        13
13.0        10
10.0         8
9.0          7
12.0         6
30.0         2
40.0         2
29.0         2
38.0         2
22.0         1
42.0         1
26.0         1
132.0        1
16.0         1
24.0         1
15.0         1
80.0         1
14.0         1
35.0         1
19.0         1
Name: QTD_RECORRENTES, dtype: int64

In [10]:
df_tudo.loc[ df_tudo["QTD_RECORRENTES"] == 1, : ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12071 entries, 0 to 29892
Data columns (total 8 columns):
LINK               12071 non-null object
INTEIRO_TEOR       12071 non-null object
DATA               12071 non-null object
RELATOR            12071 non-null object
ORGAO              12071 non-null object
RECORRENTE         12071 non-null object
DISPOSITIVO        12071 non-null object
QTD_RECORRENTES    12071 non-null float64
dtypes: float64(1), object(7)
memory usage: 848.7+ KB


In [11]:
df_filtrada = df_tudo.loc[ df_tudo["QTD_RECORRENTES"] == 1, : ]
df_filtrada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12071 entries, 0 to 29892
Data columns (total 8 columns):
LINK               12071 non-null object
INTEIRO_TEOR       12071 non-null object
DATA               12071 non-null object
RELATOR            12071 non-null object
ORGAO              12071 non-null object
RECORRENTE         12071 non-null object
DISPOSITIVO        12071 non-null object
QTD_RECORRENTES    12071 non-null float64
dtypes: float64(1), object(7)
memory usage: 848.7+ KB


## Salvamento dos dados

In [12]:
pasta = ".\\Datasets_transformados\\"
arquivo = arquivo[:-4]+"_transf.csv"
df_filtrada.to_csv( pasta+arquivo, encoding='utf-8', index=False )